<a href="https://colab.research.google.com/github/juliojesus15/SankeyDiagramMapping/blob/main/MappingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np 

students = pd.read_csv("drive/MyDrive/UCSP/Colab/DataToSankeyDiagram/dataset/students.csv")

new_columns = {
  "id": "studentId", 
  "enrollmentYear": "enrollment",
  "semester": "currentSemester",
  "program": "career",
  "dropoutStatus": "dropout"
}

students.rename(columns=new_columns, inplace=True)

students.head()

,studentId,enrollment,name,lastName,gender,career,currentSemester,dropout
0,31312b86-09b0-4afe-b63b-7c15f8c2ef71,2011-02,Dino,Raynor,male,Industrial,10,False
1,e1628900-7fff-45e0-9a66-421b329f6a6c,2012-02,Jennyfer,Stamm,female,Computer Science,7,True
2,89c32c0f-76b5-43d2-abb0-c8e1a54421dc,2012-02,Lavon,Orn,female,Industrial,3,False
3,df4a906a-4792-49e4-86c5-a445c0f05262,2012-01,Vergie,Hodkiewicz,male,Computer Science,4,False
4,7f9695e9-0093-4ea1-b5a4-059a5c89de70,2011-01,Jana,Schamberger,male,Civil,9,False


In [3]:
enrollments = pd.read_csv("drive/MyDrive/UCSP/Colab/DataToSankeyDiagram/dataset/semesters.csv")
enrollments.rename(columns={"id": "timelapse"}, inplace=True)

enrollments.head()

,timelapse,studentId,semester
0,2011-02,31312b86-09b0-4afe-b63b-7c15f8c2ef71,1
1,2012-01,31312b86-09b0-4afe-b63b-7c15f8c2ef71,2
2,2012-02,31312b86-09b0-4afe-b63b-7c15f8c2ef71,3
3,2013-01,31312b86-09b0-4afe-b63b-7c15f8c2ef71,4
4,2013-02,31312b86-09b0-4afe-b63b-7c15f8c2ef71,5


In [4]:
dataset = pd.merge(students, enrollments, on='studentId')
dataset.head()

,studentId,enrollment,name,lastName,gender,career,currentSemester,dropout,timelapse,semester
0,31312b86-09b0-4afe-b63b-7c15f8c2ef71,2011-02,Dino,Raynor,male,Industrial,10,False,2011-02,1
1,31312b86-09b0-4afe-b63b-7c15f8c2ef71,2011-02,Dino,Raynor,male,Industrial,10,False,2012-01,2
2,31312b86-09b0-4afe-b63b-7c15f8c2ef71,2011-02,Dino,Raynor,male,Industrial,10,False,2012-02,3
3,31312b86-09b0-4afe-b63b-7c15f8c2ef71,2011-02,Dino,Raynor,male,Industrial,10,False,2013-01,4
4,31312b86-09b0-4afe-b63b-7c15f8c2ef71,2011-02,Dino,Raynor,male,Industrial,10,False,2013-02,5


In [43]:
def create_container(keys):
  container = {}
  # Adding each enrollment time as a key
  for id in enrollment_ids:
    container[ id ] = {}
  return container

def mapping_students(dataframe, container):
  for register in dataframe.index:
    id = dataframe['timelapse'][ register ]
    enrollment = dataframe['enrollment'][ register ]
    studentId = dataframe['studentId'][ register ]
    semester = dataframe['semester'][ register ]

    # Checking if it is the first enrollment (N = new)
    key = str(semester) + "N" if id == enrollment else str(semester)

    if key in container[ id ] :
      container[ id ][ key ]['students'].append( studentId )
    else:
      container[ id ] = { 
        **container[ id ],
        key : { 
          'semesterId': key, 
          'students': [ studentId ] 
        } 
      } 
  return container

def intersectArrays(array1, array2):
  output = set(array1) & set(array2)
  output = list(output)

  found = True if len(output) > 0 else False
  return found, output

In [44]:
enrollment_ids = enrollments.timelapse.unique()

enrollment_ids = np.array(enrollment_ids)

enrollment_ids.sort()

CONTAINER = create_container(enrollment_ids)
CONTAINER = mapping_students(dataset, CONTAINER)

#CONTAINER

In [20]:
begin_timelapse = '2011-01'
end_timelapse = '2013-02'

condition = (begin_timelapse <= enrollment_ids) & (enrollment_ids<=end_timelapse)

new_timelapse = enrollment_ids[ condition ]
new_timelapse

array(['2011-01', '2011-02', '2012-01', '2012-02', '2013-01', '2013-02'],
      dtype=object)

In [53]:
NODES = []
LINKS = []
roadmap = {
  '0': [],
  '1N': ['1','2','0'],
  '1':  ['1','2','0'],
  '2':  ['2','3','0'],
  '3':  ['3','4','0'],
  '4':  ['4','5','0'],
  '5':  ['5','6','0'],
  '6':  ['6','7','0'],
  '7':  ['7','8','0'],
  '8':  ['8','9','0'],
  '9':  ['9','10','0'],
  '10': ['10','0'],
}

num_total_students = len(CONTAINER[ begin_timelapse ]['1N']['students'])


#num_total_students
def generate_links(timelapse_begin, timelapse_end, container):
  source = container[ timelapse_begin ]
  target = container[ timelapse_end ]
    
  for key in source:
    
    neighborhood = roadmap[ key ]
    for neighbor in neighborhood:
      #print("from: ", source[ key ]['students'])

      # checking if neighbor exist in neighborhood
      exist_neighbor = True if neighbor in target else False

      if exist_neighbor:

        found, output = intersectArrays(source[ key ]['students'], target[ neighbor ]['students'])
        print(key, " => ", neighbor)
        print(output)
      #if neighbor in data_to:
      #  print(key, " => ",neighbor) 
      #  print(data_to[neighbor]['students'])
    print("===================")


  #print("From: ", new_timelapse[ index ], " => To: ", new_timelapse[ index + 1 ] )

generate_links('2011-01', '2011-02', CONTAINER)

1N  =>  1
['a3e425eb-98be-4e11-89b3-5c268f4ec238']
1N  =>  2
['315a76ab-a5d6-41f2-a1fd-df9a7088ca20', '7f9695e9-0093-4ea1-b5a4-059a5c89de70', '73b651ab-21de-4929-9439-c8149178d608', '81c5e323-f3c3-46e6-8f96-ef883f852b6c']
3  =>  3
['2f315a97-7403-423f-a8fd-8162754cb285']
3  =>  4
['12ce3c72-efa2-468f-abee-ecbae581ccb4']
2  =>  2
[]
2  =>  3
['427580a2-4a16-4108-b56d-53738aede3cc']
